In [18]:
#Import Libraries and Read the data
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
#import xgboost
from sklearn.model_selection import train_test_split

In [26]:
#import sys
#!{sys.executable} -m pip install xgboost

# install worked but still can't import? 

# Read in data

In [38]:
taxa = pd.read_csv("/fs/ess/PAS1695/projects/exorien/data/drake-output/2022-03-16/2022-03-16_RA-with-taxonomy.csv")
taxa = taxa.drop(columns=['microbe', 'Taxonomy', 'domain', 'kingdom', 'phylum', 'class', 'order','family', 'species'])
taxa.head()

,sample,genus,exo.ra
0,SL204951,g__Gammaretrovirus,1.588590e-09
1,SL204955,g__Gammaretrovirus,1.588577e-09
2,SL204959,g__Gammaretrovirus,1.580972e-09
3,SL204960,g__Gammaretrovirus,1.580962e-09
4,SL204966,g__Gammaretrovirus,1.580976e-09


In [36]:
clin = pd.read_csv("/fs/ess/PAS1695/projects/exorien/data/IO_NOVA_clin.txt",  sep = "\t", header=None)
clin.columns = ["Sample_ID", "Cancer_Type", "OS_Event", "OS", "Response"]
clin = clin[clin['Cancer_Type'] == 'Melanoma'] 
clin = clin.drop(columns=["Cancer_Type", "OS_Event", "OS"])
clin.head()

,Sample_ID,Response
79,SL399219,1
80,SL399360,1
81,SL372342,1
82,SL362825,1
83,SL362853,1


# Transform: long to wide

In [42]:
# summarize
taxa_mel = taxa[taxa['sample'].isin(clin['Sample_ID'])]
taxaSum = taxa_mel.groupby(['sample', 'genus'], as_index=False)['exo.ra'].sum()

# long to wide
taxaWide = pd.pivot(taxaSum, index='sample', columns='genus',values='exo.ra')
taxaWide.head()

genus,g__Abiotrophia,g__Abyssicoccus,g__Acetilactobacillus,g__Acetivibrio,g__Acetomicrobium,g__Acholeplasma,g__Achromobacter,g__Acidianus,g__Acidibrevibacterium,g__Acidiferrobacter,...,g__unclassified-s__Mycobacterium phage LilMcDreamy,g__unclassified-s__Mycobacterium phage Pinnie,g__unclassified-s__Mycobacterium phage Saguaro,g__unclassified-s__Orpheovirus IHUMI-LCC2,g__unclassified-s__Providencia phage Kokobel1,g__unclassified-s__Pseudomonas phage PspYZU01,g__unclassified-s__Streptococcus phage 5093,g__unclassified-s__Yersinia phage phiR1-37,g__unclassified-s__endosymbiont of unidentified scaly snail isolate Monju,g__unclassified-s__uncultured crAssphage
sample,,,,,,,,,,,,,,,,,,,,,
SL251102,1.607202e-09,1.579746e-09,1.576725e-09,3.180566e-09,1.583079e-09,3.151087e-09,9.625264e-09,1.577513e-09,1.576350e-09,1.580575e-09,...,1.569211e-09,1.574733e-09,1.576702e-09,1.588750e-09,1.576221e-09,1.577429e-09,1.578461e-09,1.575811e-09,1.575125e-09,1.586441e-09
SL257956,1.607202e-09,1.579746e-09,1.576725e-09,3.180567e-09,1.583079e-09,3.151087e-09,9.625264e-09,1.577514e-09,1.576350e-09,1.580576e-09,...,1.569211e-09,1.574733e-09,1.576702e-09,1.588750e-09,1.576221e-09,1.577429e-09,1.578461e-09,1.575811e-09,1.575125e-09,1.586441e-09
SL271795,1.607205e-09,1.579750e-09,1.576728e-09,3.180573e-09,1.583082e-09,3.151094e-09,9.625283e-09,1.577517e-09,1.576353e-09,1.580579e-09,...,1.569214e-09,1.574736e-09,1.576705e-09,1.588753e-09,1.576224e-09,1.577432e-09,1.578464e-09,1.575814e-09,1.575128e-09,1.586444e-09
SL310764,1.579697e-09,1.564247e-09,1.576641e-09,3.149183e-09,1.572225e-09,3.145508e-09,9.608067e-09,1.575625e-09,1.552914e-09,1.569752e-09,...,1.576462e-09,1.562613e-09,1.576756e-09,1.567808e-09,1.577058e-09,1.573992e-09,1.577273e-09,1.575824e-09,1.569065e-09,1.580228e-09
SL310813,1.581751e-09,1.588023e-09,1.576136e-09,3.167039e-09,1.581881e-09,3.155622e-09,9.221916e-09,1.577431e-09,1.602051e-09,1.580005e-09,...,1.575820e-09,1.580558e-09,1.576471e-09,1.598604e-09,1.575857e-09,1.579521e-09,1.577862e-09,1.575901e-09,1.551818e-09,1.581905e-09


In [44]:
clin = clin.set_index('Sample_ID')

# Reduce Dimensionality 

In [45]:
# Create correlation matrix
corr_matrix = taxaWide.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.005 - why are the correlation values so low?
to_drop = [column for column in upper.columns if any(upper[column] > 0.005)]

# Drop features 
taxaWide.drop(to_drop, axis=1, inplace=True)

taxaWide.head()

genus,g__Abiotrophia
sample,
SL251102,1.607202e-09
SL257956,1.607202e-09
SL271795,1.607205e-09
SL310764,1.579697e-09
SL310813,1.581751e-09


# Split into testing and training

In [46]:
x = taxaWide
y = clin

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, shuffle=True)

# Model

In [48]:
classifiers =[]

#model1 = xgboost.XGBClassifier()
#classifiers.append(model1)

model2 = svm.SVC()
classifiers.append(model2)

model3 = tree.DecisionTreeClassifier()
classifiers.append(model3)

model4 = RandomForestClassifier()
classifiers.append(model4)

In [49]:
for clf in classifiers:
    clf.fit(x_train, y_train)
    y_pred= clf.predict(x_test)
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy of %s is %s"%(clf, acc))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix of %s is %s"%(clf, cm))

Accuracy of SVC() is 0.6666666666666666
Confusion Matrix of SVC() is [[ 0  5]
 [ 0 10]]
Accuracy of DecisionTreeClassifier() is 0.6666666666666666
Confusion Matrix of DecisionTreeClassifier() is [[ 0  5]
 [ 0 10]]
Accuracy of RandomForestClassifier() is 0.6666666666666666
Confusion Matrix of RandomForestClassifier() is [[ 0  5]
 [ 0 10]]


/apps/python/3.9-2022.05/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/slurmtmp.13052181/ipykernel_162867/38144895.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(x_train, y_train)


In [14]:
classifiers

[SVC(), DecisionTreeClassifier(), RandomForestClassifier()]

# Artifically expand dataset?

# Look at all cancer types? 